In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

# Configurar o Selenium
chrome_options = Options()
chrome_options.add_argument("--headless")  # Rodar em modo headless

# Caminho do arquivo CSV
csv_file = "artigos_exame.csv"

# Função principal para coletar dados
def coletar_dados():
    driver = webdriver.Chrome(options=chrome_options)
    dados_artigos = []

    # Verificar se o arquivo CSV já existe
    if os.path.exists(csv_file):
        df_existente = pd.read_csv(csv_file)
        titulos_existentes = set(df_existente['Título'].tolist())  # Coletar títulos existentes
    else:
        titulos_existentes = set()  # Nenhum título existe ainda

    try:
        # Acessar a página inicial
        url = 'https://exame.com/economia/'
        driver.get(url)

        # Encontrar os links dos primeiros dois artigos
        links_artigos = [
            driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/div[4]/section/div[2]/div/div[3]/div[2]/h3/a').get_attribute('href'),
            driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/div[4]/section/div[2]/div/div[1]/div[2]/h3/a').get_attribute('href'),
            driver.find_element(By.XPATH,'//*[@id="__next"]/main/div/div[4]/section/div[4]/div/div[1]/div[2]/h3/a').get_attribute('href'),
            driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/div[5]/section/div[2]/div/div[1]/div[2]/h3/a').get_attribute('href'),
            driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/div[6]/section/div[1]/div/div[3]/div/div[1]/div[2]/h3/a').get_attribute('href')
        
        
        
        
        ] 
        
        

        # Iterar sobre cada link dos artigos
        for i, link in enumerate(links_artigos):
            print(f"\nAcessando artigo {i+1}: {link}")
            driver.get(link)

            # Aguarde até o título <h1> carregar
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'h1')))

            # Processar o conteúdo da nova página com BeautifulSoup
            html_content = driver.page_source
            soup = BeautifulSoup(html_content, 'html.parser')

            # Extrair título e parágrafos
            titulo = soup.find('h1').get_text(strip=True) if soup.find('h1') else "Título não encontrado"
            paragrafos = [p.get_text(strip=True) for p in soup.find_all('p')]

            # Verificar se o título já existe
            if titulo in titulos_existentes:
                print(f"Artigo '{titulo}' já existe. Ignorando.")
                continue  # Pule para o próximo link

            # Salvar os dados em um dicionário
            dados_artigos.append({
                'Título': titulo,
                'Conteúdo': " ".join(paragrafos)
            })

            print(f"Artigo '{titulo}' processado com sucesso.")

    finally:
        driver.quit()
        print("\nNavegador fechado.")

    # Se houver novos dados, salvar no CSV
    if dados_artigos:
        df_novos = pd.DataFrame(dados_artigos)
        print("\nNovos artigos coletados:")
        print(df_novos)

        if os.path.exists(csv_file):
            df_novos.to_csv(csv_file, mode='a', index=False, header=False)
        else:
            df_novos.to_csv(csv_file, index=False)

        print("Novos dados salvos no arquivo 'artigos_exame.csv'.")
    else:
        print("Nenhum novo artigo foi encontrado.")

# Loop infinito que executa a cada 5 minutos
while True:
    print("\nIniciando nova execução...")
    coletar_dados()
    print("Aguardando 5 minutos para a próxima execução...")
    time.sleep(300)  # Pausa por 300 segundos (5 minutos)



Iniciando nova execução...

Acessando artigo 1: https://exame.com/economia/votacao-da-reforma-tributaria-e-adiada-para-terca-feira/
Artigo 'Economia' processado com sucesso.

Acessando artigo 2: https://exame.com/economia/china-planeja-deficit-orcamentario-recorde-em-2025-afirma-reuters/
Artigo 'Economia' processado com sucesso.

Acessando artigo 3: https://exame.com/economia/relatorio-ldo-2025-regras-contingenciamento-orcamento/
Artigo 'Economia' processado com sucesso.

Acessando artigo 4: https://exame.com/economia/sem-reavaliacao-de-politicas-espaco-para-despesas-livres-pode-se-esgotar-em-2030-projeta-tesouro/
Artigo 'Economia' processado com sucesso.

Acessando artigo 5: https://exame.com/economia/renegociacoes-de-dividas-de-icms-e-ipva-em-sp-somam-r-47-bilhoes-em-um-ano/
Artigo 'Economia' processado com sucesso.

Navegador fechado.

Novos artigos coletados:
     Título                                           Conteúdo
0  Economia  Acesse o melhor conteúdo do seu dia, o único q.

In [ ]:
df